In [1]:
import pandas as pd
from datetime import datetime as dt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
! ls

Batiments_vacants_VM_2016	  clean_requetes311_2022-present.csv
cleaning_and_prelim_eda_IA.ipynb  geospatial_tool.ipynb
clean_requetes311_2014-2016.csv   README.md
clean_requetes311_2016-2018.csv   territoires-administratifs-casernes
clean_requetes311_2019-2021.csv


In [3]:
data_2014_2016 = pd.read_csv('requetes311_2014-2016.csv', low_memory=False)
data_2016_2018 = pd.read_csv('requetes311_2016-2018.csv', low_memory=False)
data_2019_2021 = pd.read_csv('requetes311_2019-2021.csv', low_memory=False)
data_2022_present = pd.read_csv('requetes311_2022-present.csv', low_memory=False)

In [4]:
NAMES = ['2014-2016', '2016-2018', '2019-2021', '2022-present']
DATASETS = [data_2014_2016, data_2016_2018, data_2019_2021, data_2022_present]
DATA_DICT = dict(zip(NAMES,DATASETS))

In [5]:
DATA_DICT.keys()

# CLEAN: remove all rows without geo-spacial data
SAVE output csv

In [6]:
for key in DATA_DICT.keys():
    print(f'processing {key}...')
    data = DATA_DICT[key]
    data.dropna(subset=['LOC_LONG'], inplace=True)
    
    data = data[data['NATURE'] != 'Commentaire']
    
    data_clean = data.copy()
    
    data_clean.to_csv(f"clean_{key}")
    print(f'clean_{key} complete')

# LOAD cleaned data (if needed)

In [7]:
# data_2014_2016 = pd.read_csv('clean_requetes311_2014-2016.csv', low_memory=False)
# data_2016_2018 = pd.read_csv('clean_requetes311_2016-2018.csv', low_memory=False)
# data_2019_2021 = pd.read_csv('clean_requetes311_2019-2021.csv', low_memory=False)
# data_2022_present = pd.read_csv('clean_requetes311_2022-present.csv', low_memory=False)

In [8]:
NAMES = ['2014-2016', '2016-2018', '2019-2021', '2022-present']
DATASETS = [data_2014_2016, data_2016_2018, data_2019_2021, data_2022_present]
DATA_DICT = dict(zip(NAMES,DATASETS))

In [9]:
for key in DATA_DICT:
    print(f' {key}:')
    data = DATA_DICT[key]
    data.value_counts('NATURE')

 2014-2016:


NATURE
Requete    1060340
Plainte      14348
Name: count, dtype: int64

 2016-2018:


NATURE
Requete    1172488
Plainte      15374
Name: count, dtype: int64

 2019-2021:


NATURE
Requete    1014315
Plainte      18486
Name: count, dtype: int64

 2022-present:


NATURE
Requete    1013817
Plainte      17308
Name: count, dtype: int64

# concetenate all datasets

In [10]:
full_data = pd.concat([DATA_DICT[key] for key in DATA_DICT.keys()])

In [11]:
data = full_data

# change time data to datetime format

In [12]:
data['DDS_DATE_CREATION'] = pd.to_datetime(data['DDS_DATE_CREATION'])
data['DATE_DERNIER_STATUT'] = pd.to_datetime(data['DATE_DERNIER_STATUT'])

# EDA revealed numerous missing closure date between 2014-2015 - so remove rows with no DATE_DERNIER_STATUT time 

In [13]:
data.dropna(subset=['DATE_DERNIER_STATUT'], inplace=True)

# create column Termination time - response time

In [14]:
data['TERMINATION_TIME'] = (data['DATE_DERNIER_STATUT'] - data['DDS_DATE_CREATION'])

In [15]:
data[['DATE_DERNIER_STATUT','DDS_DATE_CREATION']].head(5)

,DATE_DERNIER_STATUT,DDS_DATE_CREATION
685640,2018-11-01 06:51:40,2016-11-29 10:03:07
685641,2018-01-17 10:37:13,2016-09-01 14:51:04
685642,2017-11-07 08:04:37,2016-10-21 14:49:39
685643,2016-06-23 14:23:15,2016-06-23 11:25:58
685644,2016-05-16 09:59:02,2016-04-25 11:22:37


#  add month (or issue creation) to analyze seasonality

In [19]:
data['MONTH'] = data['DDS_DATE_CREATION'].apply(lambda x: x.month)

# save dataset as csv

In [21]:
data.reset_index().to_csv('processed_311data.csv')

In [ ]:
# HAZARDS = [
#     "Feux de circulation - Entretien",         #'Traffic lights - Maintenance',
#     "Branche tombée",                          # 'Fallen branch',
#     "Débris sur la voie publique",             # 'Debris on the public highway',
#     "Fuite d'eau",                             # 'Water leak',
#     "Trottoir glissant ",                      # 'Slippery sidewalk',
#     "Chaussée glissante",                      # 'Slippery road',
#     "Fermeture d'une conduite d'eau - Urgence",# 'Closure of a water pipe - Emergency'
# ]

In [ ]:
# data.loc[data['ACTI_NOM'].isin(HAZARDS),'TERMINATION_TIME'].describe()